In [ ]:
# Import libraries

import os
import json
from uuid import uuid4
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_cohere import CohereEmbeddings
from langchain.schema import Document
from dotenv import load_dotenv
load_dotenv()

In [ ]:
def read_user_details(directory):
    with open(directory, 'r') as f:
        data = json.load(f)
        return data
    
user_profiles = read_user_details('users.json')
print(user_profiles)

In [ ]:
# Create function to convert profile to text
def profile_to_text(profile):
    interests = ", ".join(profile.get("Interests", []))
    likes = ", ".join(profile.get("Likes", []))
    dislikes = ", ".join(profile.get("Dislikes", []))
    hobbies = ", ".join(profile.get("Hobbies", []))
    
    return (
        f"Age: {profile['Age']}, "
        f"{profile['Gender']}, "
        f"{interests}, {likes}, {dislikes}, "
        f"{profile['Occupation']}, "
        f"{hobbies}, {profile['Looking For']}"
    )

documents = [
    Document(
        page_content=profile_to_text(profile),
        metadata={
            "Name": profile['Name'],
            "Location": profile['Location'],
            "Gender": profile['Gender'],
            "Looking For": profile['Looking For']
        }
    )
    for profile in user_profiles
]
documents

In [ ]:
embeddings_model = CohereEmbeddings(cohere_api_key=os.environ['COHERE_API_KEY'], model='embed-english-v3.0')
embeddings_model

In [ ]:
# Vector Search DB In Pinecone
pc = Pinecone(api_key=os.environ['PINECONE_API_KEY'])
index = pc.Index(os.environ['INDEX_NAME'])

uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store = PineconeVectorStore(index=index, embedding=embeddings_model)
vector_store.add_documents(ids=uuids, documents=documents)

In [ ]:
user = {
  "id": 17,
  "Name": "Oliver Green",
  "Age": 27,
  "Location": "Auckland, New Zealand",
  "Gender": "Male",
  "Interests": [
    "Surfing",
    "Hiking",
    "Photography"
  ],
  "Likes": [
    "Seafood",
    "Adventure sports",
    "Nature documentaries"
  ],
  "Dislikes": [
    "Crowds",
    "Fast food"
  ],
  "Occupation": "Marine biologist",
  "Hobbies": [
    "Traveling",
    "Cooking"
  ],
  "Looking For": "Casual dating"
}

user_text = profile_to_text(user)

results = vector_store.similarity_search_with_score(
    user_text, 
    k=4,
    filter={
      "Looking For": user["Looking For"],
      "Gender": "Female"
    }
)

results